##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
from google import genai
from google.genai import types

import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [6]:
MODEL_ID = "gemini-2.0-flash"

client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
def add(a: float, b: float):
    """returns a+b."""
    return a + b

def sub(a: float, b: float):
    """returns a-b."""
    return a - b

def mul(a: float, b: float):
    """returns a*b."""
    return a * b

def div(a: float, b: float):
    """returns a/b."""
    return a / b


In [7]:
operational_tools = [add, sub, mul, div]

chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": operational_tools,
    }
)

In [8]:
response = chat.send_message("57마리의 고양이가 각각 44개의 장갑을 가지고 있다면, 전부 장갑은 몇개인가?")
response.text

'전부 2508개의 장갑이 있습니다.'

In [10]:
from IPython.display import Markdown, display

for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(f"Function call: {part.function_call}")
        if part.function_response:
            print(f"Function response: {part.function_response}")
    print("-"*80)

### user:

57마리의 고양이가 각각 44개의 장갑을 가지고 있다면, 전부 장갑은 몇개인가?

--------------------------------------------------------------------------------


### model:

Function call: id=None args={'b': 44, 'a': 57} name='mul'
--------------------------------------------------------------------------------


### user:

Function response: id=None name='mul' response={'result': 2508}
--------------------------------------------------------------------------------


### model:

전부 2508개의 장갑이 있습니다.

--------------------------------------------------------------------------------


In general the state diagram is:

![Function calling diagram](https://camo.githubusercontent.com/fa0b343427abf76115df71c5548c20b5ce65bbe0868b8eaa03c04c789e176895/68747470733a2f2f636f64656c6162732e646576656c6f706572732e676f6f676c652e636f6d2f7374617469632f636f64656c6162732f67656d696e692d66756e6374696f6e2d63616c6c696e672f696d672f67656d696e692d66756e6374696f6e2d63616c6c696e672d6f766572766965775f313434302e706e67)

The model can respond with multiple function calls before returning a text response, and function calls come before the text response.

## Manual function calling

In [13]:
def find_movies(description: str, location: str):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]

def find_theaters(location: str, movie: str):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]

def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """

    return ["10:00", "11:00"]

In [18]:
theater_functions = [find_movies, find_theaters, get_showtimes]

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Which theaters in Mountain View show the Barbie movie?",
    config={
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)


In [22]:
for part in response.candidates[0].content.parts:
    print(part.function_call)

id=None args={'location': 'Mountain View, CA', 'movie': 'Barbie'} name='find_theaters'


In [23]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    # Find the function object from the list based on the function name
    for func in functions:
        if func.__name__ == function_name:
            return func(**function_args)

In [24]:
part = response.candidates[0].content.parts[0]

if part.function_call:
    result = call_function(part.function_call, theater_functions)

In [25]:
print(result)

['Googleplex 16', 'Android Theatre']


In [33]:
# Build the message history
messages = [
    genai.types.Content(
        role="user",
        parts=[genai.types.Part(text="Which theaters in Mountain View show the Barbie movie?")],
    ),
    genai.types.Content(
        role="model",
        parts=[part]
    ),
    {
        "role": "user", 
        "parts": [
            {
                "function_response": {
                    "response": {"output": result},
                    "name": part.function_call.name}
            }
        ]
    },
]

In [34]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)

In [35]:
print(response.text)

Theaters in Mountain View that show the Barbie movie are Googleplex 16 and Android Theatre.


## Function calling chain

In [41]:
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": theater_functions,
    }
)

response = chat.send_message(
    "Which comedy movies are shown tonight (01/01/2025) in Mountain view, in which cinema and at what time?"
)



In [46]:
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(f"Function call: {part.function_call}")
        if part.function_response:
            print(f"Function response: {part.function_response}")
    print("-"*80)

### user:

Which comedy movies are shown tonight (01/01/2025) in Mountain view, in which cinema and at what time?

--------------------------------------------------------------------------------


### model:

Function call: id=None args={'location': 'Mountain View, CA', 'description': 'comedy'} name='find_movies'
--------------------------------------------------------------------------------


...

### user:

Function response: id=None name='get_showtimes' response={'result': ['10:00', '11:00']}
--------------------------------------------------------------------------------


### model:

Oppenheimer is playing at Googleplex 16 at 10:00 and 11:00 and at Android Theatre at 10:00 and 11:00.


--------------------------------------------------------------------------------


## Parallel function calls

Gemini API는 한 번에 여러 함수를 호출할 수 있습니다. 이는 작업 완료를 위해 여러 함수 호출이 독립적으로 수행되는 시나리오에 적합합니다.

먼저 도구를 설정하세요. 위 동영상 예시와 달리, 이 함수들은 호출 시 서로의 입력이 필요하지 않으므로 병렬 호출에 적합합니다.

In [80]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [86]:
house_fns = [power_disco_ball, start_music, dim_lights]

chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": house_fns,        
    }
)

In [87]:
response = chat.send_message("Turn this place into a party! Play music, spin disco ball and dim the lights!")

Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 50%


In [88]:
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(f"Function call: {part.function_call}")
        if part.function_response:
            print(f"Function response: {part.function_response}")
    print("-"*80)

### user:

Turn this place into a party! Play music, spin disco ball and dim the lights!

--------------------------------------------------------------------------------


### model:

Let's get this party started!


...

### user:

Function response: id=None name='start_music' response={'result': 'Never gonna give you up.'}
Function response: id=None name='power_disco_ball' response={'result': True}
Function response: id=None name='dim_lights' response={'result': True}
--------------------------------------------------------------------------------


### model:

Alright! I'm playing "Never gonna give you up.", the disco ball is spinning, and the lights are dimmed. Let's party!


--------------------------------------------------------------------------------


## Related examples

Check those examples using function calling to give you more ideas on how to use that very useful feature:

* [Barista Bot](https://github.com/google-gemini/cookbook/blob/e0bac2900f5f79e1920927fe524229486f44ed78/examples/Agents_Function_Calling_Barista_Bot.ipynb), an agent to order coffee
* [Browser-as-a-tool](https://github.com/google-gemini/cookbook/blob/e0bac2900f5f79e1920927fe524229486f44ed78/examples/Browser_as_a_tool.ipynb), using function calling to call a web-browser.
* Using function calling to [re-rank seach results](https://github.com/google-gemini/cookbook/blob/e0bac2900f5f79e1920927fe524229486f44ed78/examples/Search_reranking_using_embeddings.ipynb).
* [Using tools with the Live API](https://github.com/google-gemini/cookbook/blob/e0bac2900f5f79e1920927fe524229486f44ed78/quickstarts/Get_started_LiveAPI_tools.ipynb), using function calling and other tools with the Live APIs.